In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary


In [2]:
# Get the number of schools in the dataset and store that number
num_of_schools = len(school_data_complete["School ID"].unique())

In [3]:
# Get the number of students in the dataset and store that number
num_of_students = len(school_data_complete["Student ID"].unique())

In [4]:
# Summarize budget by school and retain first value for budget for each school
budget_by_school = pd.DataFrame(school_data_complete.groupby(["School ID"])["budget"].first())

# Sum budgets across all schools and store that number
total_budget = budget_by_school["budget"].sum()

In [5]:
# Store average math and reading scores across all students
scores = school_data_complete[["Student ID","reading_score","math_score"]]
avg_math = scores["math_score"].mean()
avg_reading = scores["reading_score"].mean()

In [6]:
# Calculate percentage of students passing math
scores_math = pd.DataFrame(scores.loc[scores["math_score"] >= 70,["Student ID","math_score"]])
passing_math = len(scores_math["Student ID"]) / num_of_students * 100

In [7]:
# Calculate percentage of students passing reading
scores_reading = pd.DataFrame(scores.loc[scores["reading_score"] >= 70,["Student ID","reading_score"]])
passing_reading = len(scores_reading["Student ID"]) / num_of_students * 100

In [8]:
# Calculate percentage of students passing both math and reading
scores_overall = pd.DataFrame(scores.loc[(scores["reading_score"] >= 70) & (scores["math_score"] >= 70),
                                         ["Student ID","reading_score","math_score"]])
passing_overall = len(scores_overall["Student ID"]) / num_of_students * 100

In [9]:
# Create dataframe summarizing district-wide numbers
district_summary = pd.DataFrame({"Number of Schools" : [num_of_schools],
                                "Number of Students" : [num_of_students],
                                "Total Budget" : [total_budget],
                                "Average Math Score" : [avg_math],
                                "Average Reading Score" : [avg_reading],
                                "Percentage of Students Passing Math" : [passing_math],
                                "Percentage of Students Passing Reading" : [passing_reading],
                                "Percentage of Students Passing Math and Reading" : [passing_overall]})

In [10]:
# Apply formats to the values in the dataframe
district_summary["Number of Students"] = district_summary["Number of Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:.2f}".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:.2f}".format)
district_summary["Percentage of Students Passing Math"] = district_summary["Percentage of Students Passing Math"].map("{:.2f}%".format)
district_summary["Percentage of Students Passing Reading"] = district_summary["Percentage of Students Passing Reading"].map("{:.2f}%".format)
district_summary["Percentage of Students Passing Math and Reading"] = district_summary["Percentage of Students Passing Math and Reading"].map("{:.2f}%".format)

In [11]:
# display the district-wide summary
district_summary

,Number of Schools,Number of Students,Total Budget,Average Math Score,Average Reading Score,Percentage of Students Passing Math,Percentage of Students Passing Reading,Percentage of Students Passing Math and Reading
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

In [12]:
# Create separate dataframe from original school data
school_data_df = pd.DataFrame(school_data_complete)

In [13]:
# Create Passing variables based on whether each student is passing
school_data_df["% Passing Reading"] = 0
school_data_df.loc[school_data_df["reading_score"] >=70, "% Passing Reading"] = 1
school_data_df["% Passing Math"] = 0
school_data_df.loc[school_data_df["math_score"] >=70, "% Passing Math"] = 1
school_data_df["% Overall Passing"] = 0
school_data_df.loc[(school_data_df["math_score"] >=70) & (school_data_df["reading_score"] >=70),
                         "% Overall Passing"] = 1

# Rename "school_name" variable 
school_data_df = school_data_df.rename(columns={'school_name' : 'School Name'})

In [14]:
# Calculate average scores and the percentage of students passing by school
scores_by_school = pd.DataFrame(school_data_df.groupby(["School Name"])
                                [["reading_score","math_score",
                                 "% Passing Reading", "% Passing Math","% Overall Passing"]].mean())

In [15]:
# Summarize by school and retain first instance of type, budget, and size
stats_by_school = pd.DataFrame(school_data_df.groupby(["School Name"])[["type","budget","size"]].first())

# Calculate the budget per student for each school
stats_by_school["Per Student Budget"] = (stats_by_school["budget"] / stats_by_school["size"]).astype(int)

In [16]:
# Merge school stats and school scores datasets together by School Name
school_summary = pd.merge(stats_by_school,scores_by_school,on="School Name")

In [17]:
# Rename variables
school_summary = school_summary.rename(columns={"type" : "School Type",
                                               "budget" : "Total School Budget",
                                               "size" : "Total Students",
                                               "reading_score" : "Average Reading Score",
                                               "math_score" : "Average Math Score"})

# Multiply percentage columns by 100
school_summary['% Passing Reading'] = school_summary['% Passing Reading'] * 100
school_summary['% Passing Math'] = school_summary['% Passing Math'] * 100
school_summary['% Overall Passing'] = school_summary['% Overall Passing'] * 100


In [18]:
# Create a copy of the dataframe to apply formatting
school_summary_format = pd.DataFrame(school_summary).copy()

In [19]:
# Apply formatting
school_summary_format["Total School Budget"] = school_summary_format["Total School Budget"].map("${:,}".format)
school_summary_format["Per Student Budget"] = school_summary_format["Per Student Budget"].map("${:,}".format)
school_summary_format["Average Reading Score"] = school_summary_format["Average Reading Score"].map("{:.2f}".format)
school_summary_format["Average Math Score"] = school_summary_format["Average Math Score"].map("{:.2f}".format)
school_summary_format["% Passing Reading"] = (school_summary_format["% Passing Reading"]).map("{:.2f}%".format)
school_summary_format["% Passing Math"] = (school_summary_format["% Passing Math"]).map("{:.2f}%".format)
school_summary_format["% Overall Passing"] = (school_summary_format["% Overall Passing"]).map("{:.2f}%".format)

In [20]:
# Display table
school_summary_format[["School Type", "Total Students", "Total School Budget", "Per Student Budget", 
                "Average Math Score", "Average Reading Score", "% Passing Math", 
                "% Passing Reading", "% Overall Passing"]]

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928",$628,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,2739,"$1,763,916",$644,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,4761,"$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%


## Top Performing Schools (By % Overall Passing)

In [21]:
# Display top performing schools based on percentage passing overall
best_performing = pd.DataFrame(school_summary_format)
best_performing.sort_values(by=["% Overall Passing"],ascending=False).head()

,School Type,Total School Budget,Total Students,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,"$1,081,356",1858,$582,83.98,83.06,97.04%,94.13%,91.33%
Thomas High School,Charter,"$1,043,130",1635,$638,83.85,83.42,97.31%,93.27%,90.95%
Griffin High School,Charter,"$917,500",1468,$625,83.82,83.35,97.14%,93.39%,90.60%
Wilson High School,Charter,"$1,319,574",2283,$578,83.99,83.27,96.54%,93.87%,90.58%
Pena High School,Charter,"$585,858",962,$609,84.04,83.84,95.95%,94.59%,90.54%


## Bottom Performing Schools (By % Overall Passing)

In [22]:
# Display worst performing schools based on percentage passing overall
bottom_performing = pd.DataFrame(school_summary_format)
bottom_performing.sort_values(by=["% Overall Passing"],ascending=True).head()

,School Type,Total School Budget,Total Students,Per Student Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,"$2,547,363",3999,$637,80.74,76.84,80.22%,66.37%,52.99%
Figueroa High School,District,"$1,884,411",2949,$639,81.16,76.71,80.74%,65.99%,53.20%
Huang High School,District,"$1,910,635",2917,$655,81.18,76.63,81.32%,65.68%,53.51%
Hernandez High School,District,"$3,022,020",4635,$652,80.93,77.29,80.86%,66.75%,53.53%
Johnson High School,District,"$3,094,650",4761,$650,80.97,77.07,81.22%,66.06%,53.54%


## Math Scores by Grade

In [23]:
# Create new dataframe for analyzing math scores by grade
math_scores = pd.DataFrame(school_data_complete[["school_name","grade","math_score"]])

# Rename 'school_name' variable
math_scores = math_scores.rename(columns={'school_name' : 'School Name'})

In [24]:
# Create grade columns and assign mass score to the column corresponding to the correct grade
math_scores.loc[math_scores["grade"] == "9th", "9th"] = math_scores["math_score"]
math_scores.loc[math_scores["grade"] == "10th", "10th"] = math_scores["math_score"]
math_scores.loc[math_scores["grade"] == "11th", "11th"] = math_scores["math_score"]
math_scores.loc[math_scores["grade"] == "12th", "12th"] = math_scores["math_score"]

In [25]:
# Summarize average math schores by school
math_scores = math_scores.groupby(["School Name"])[["9th","10th","11th","12th"]].mean()

In [26]:
# Apply formatting
math_scores["9th"] = math_scores["9th"].map("{:.2f}".format)
math_scores["10th"] = math_scores["10th"].map("{:.2f}".format)
math_scores["11th"] = math_scores["11th"].map("{:.2f}".format)
math_scores["12th"] = math_scores["12th"].map("{:.2f}".format)

# Display results
math_scores

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

In [27]:
# Create new dataframe for analyzing math scores by grade
reading_scores = pd.DataFrame(school_data_complete[["school_name","grade","reading_score"]])

# Rename 'school_name' variable
reading_scores = reading_scores.rename(columns={'school_name' : 'School Name'})

In [28]:
# Create grade columns and assign mass score to the column corresponding to the correct grade
reading_scores.loc[reading_scores["grade"] == "9th", "9th"] = reading_scores["reading_score"]
reading_scores.loc[reading_scores["grade"] == "10th", "10th"] = reading_scores["reading_score"]
reading_scores.loc[reading_scores["grade"] == "11th", "11th"] = reading_scores["reading_score"]
reading_scores.loc[reading_scores["grade"] == "12th", "12th"] = reading_scores["reading_score"]

In [29]:
# Summarize average math schores by school
reading_scores = reading_scores.groupby(["School Name"])[["9th","10th","11th","12th"]].mean()

In [30]:
# Apply formatting
reading_scores["9th"] = reading_scores["9th"].map("{:.2f}".format)
reading_scores["10th"] = reading_scores["10th"].map("{:.2f}".format)
reading_scores["11th"] = reading_scores["11th"].map("{:.2f}".format)
reading_scores["12th"] = reading_scores["12th"].map("{:.2f}".format)

# Display results
reading_scores

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

In [31]:
# Create dataframe for scores by spending analysis
by_spending = pd.DataFrame(school_summary)

In [32]:
# Create bins, labels, and slice variable 'Per Student Budget'
bins = [0,585,630,645,680]

labels = ['<$585', '$585-630', '$630-645', '$645-680']

by_spending['Spending Ranges (Per Student)'] = pd.cut(by_spending['Per Student Budget'],bins=bins,labels=labels)

In [33]:
# Summarize scores and passing percentages by spending rage per student
by_spending = by_spending.groupby(['Spending Ranges (Per Student)'])[['Average Math Score', 'Average Reading Score',
                                                       '% Passing Math', '% Passing Reading', 
                                                       '% Overall Passing']].mean()

In [34]:
# Apply formatting
by_spending['Average Math Score'] = by_spending['Average Math Score'].map("{:.2f}".format)
by_spending['Average Reading Score'] = by_spending['Average Reading Score'].map("{:.2f}".format)
by_spending['% Passing Math'] = by_spending['% Passing Math'].map("{:.2f}%".format)
by_spending['% Passing Reading'] = by_spending['% Passing Reading'].map("{:.2f}%".format)
by_spending['% Overall Passing'] = by_spending['% Overall Passing'].map("{:.2f}%".format)

# Display table
by_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46%,96.61%,90.37%
$585-630,81.90,83.16,87.13%,92.72%,81.42%
$630-645,78.52,81.62,73.48%,84.39%,62.86%
$645-680,77.00,81.03,66.16%,81.13%,53.53%


Perhaps counterintuitively, higher rates of passing in math, reading, and overall were observed in schools with smaller budgets per student in general. As the spending per student increased on average, passing rates decreased. This is likely driven by other factors (e.g., larger schools may have disproportionarly large budgets which increases the spending per student, but may decrease outcomes per student). This trend is not sufficient to suggest that schools should decrease their budgets in order to improve passing rates. 

## Scores by School Size

In [35]:
# Create dataframe for scores by school size
by_size = pd.DataFrame(school_summary)

In [36]:
# Create bins, labels, and slice variable 'Total Students'
bins = [0,1000,2000,5000]

labels = ['Small (<1000)','Medium (1000-2000)','Large (2000-5000)']

by_size['School Size'] = pd.cut(by_size['Total Students'],bins=bins,labels=labels)

In [37]:
# Summarize scores and passing percentages by school size
by_size = by_size.groupby(['School Size'])[['Average Math Score','Average Reading Score',
                                 '% Passing Math','% Passing Reading',
                                 '% Overall Passing']].mean()

In [38]:
# Apply formatting
by_size['Average Math Score'] = by_size['Average Math Score'].map("{:.2f}".format)
by_size['Average Reading Score'] = by_size['Average Reading Score'].map("{:.2f}".format)
by_size['% Passing Math'] = by_size['% Passing Math'].map("{:.2f}%".format)
by_size['% Passing Reading'] = by_size['% Passing Reading'].map("{:.2f}%".format)
by_size['% Overall Passing'] = by_size['% Overall Passing'].map("{:.2f}%".format)

# Display table
by_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,89.88%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,90.62%
Large (2000-5000),77.75,81.34,69.96%,82.77%,58.29%


While the difference in average outcomes between students in small and medium-sized schools is relatively immaterial (with high rates of passing in math, reading, and overall), the rates of passing drop substantially for the average student at a large school. 

## Scores by School Type

In [39]:
# Summarize scores and passing percentages by school type
by_type = pd.DataFrame(school_summary)

by_type = by_type.groupby('School Type')[['Average Math Score','Average Reading Score',
                                 '% Passing Math','% Passing Reading',
                                 '% Overall Passing']].mean()

In [40]:
# Apply formatting
by_type['Average Math Score'] = by_type['Average Math Score'].map("{:.2f}".format)
by_type['Average Reading Score'] = by_type['Average Reading Score'].map("{:.2f}".format)
by_type['% Passing Math'] = by_type['% Passing Math'].map("{:.2f}%".format)
by_type['% Passing Reading'] = by_type['% Passing Reading'].map("{:.2f}%".format)
by_type['% Overall Passing'] = by_type['% Overall Passing'].map("{:.2f}%".format)

# Display table
by_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%
